## We solve the MHD toy model with PINN

### The evolution equations are:

$$
\partial_t B^i = \nabla_j (v^j B^i - v^i B^j) \;\;\;\; \nabla_i B^i = 0
$$

The vector field $v^i$ is given. 

And we impose Dirichlet boundary conditions.

The initial data is: 

\begin{align*}
B_1 &= \partial_y \phi \\
B_2 &= -\partial_x \phi
\end{align*}
with (here we use xmin=0, xmax = 1, same for y)

$$
\phi(x,y) = (x*(x-1)*y*(y-1))^2
$$

The velocity field is time-independent and given by:

\begin{align*}
v1(t,x,y) &= \sin(\pi*x)*\cos(\pi*y) \\
v2(t,x,y) &= \cos(\pi*x)*\sin(\pi*y) \\
\end{align*}



In [ ]:
#import Pkg
#Pkg.add(url="https://github.com/psy3nt1st/Optim.jl.git")
using Optim
using NeuralPDE
#using Optimization
using OptimizationOptimJL
using Roots
using LineSearches
using ModelingToolkit, IntervalSets 
using IntervalSets
using Plots, Printf
using Lux, LuxCUDA, ComponentArrays, Random
using JLD2, LinearAlgebra 

const gpud = gpu_device()


@parameters t, x, y
@variables B1(..), B2(..)
Dx = Differential(x)
Dy = Differential(y)
Dt = Differential(t)

In [ ]:
# -------------------------------------------------------------------
# Configuración
# -------------------------------------------------------------------


if true
config = Dict(
    :N_input => 3,          # [t , x, y]
    :N_neurons => 64,
    :N_layers => 4,
    :N_output => 2, 
    :N_points => 15_000,     # puntos de colisión (x,t)
    :BCS_points => 15_000,
    :Minibatch => 100,
    :xmin => 0.0,
    :xmax => 1.0,           # = L dominio espacial
    :ymin => 0.0,
    :ymax => 1.0,           # = L dominio espacial
    :tmin => 0.0,           # t_min
    :tmax => 4.0,           # t_max
    #:optimizer => BFGS(),
    :optimizer => SSBroyden(),
    :maxiters => 15_000,
    # for the initial data
    :A => 1.0,
    :p => 2,
    :c => 1.0,
    # for discretizations
    :dx => [0.1, 0.1, 0.1]
)


else 
config = Dict(
    :N_input => 3,          # [t , x, y]
    :N_neurons => 64,
    :N_layers => 4,
    :N_output => 2, 
    :N_points => 15_000,     # puntos de colisión (x,t)
    :BCS_points => 15_000,
    :Minibatch => 100,
    :xmin => 0.0,
    :xmax => 1.0,           # = L dominio espacial
    :ymin => 0.0,
    :ymax => 1.0,           # = L dominio espacial
    :tmin => 0.0,           # t_min
    :tmax => 4.0,           # t_max
    #:optimizer => BFGS(),
    :optimizer => SSBroyden(),
    :maxiters => 10_000,
    # for the initial data
    :A => 1.0,
    :p => 2,
    :c => 1.0,
    # for discretizations
    :dx => [0.1, 0.1, 0.1]
)
end

@show config

In [ ]:
# Space and time domains

@show domains = [t ∈ Interval(config[:tmin], config[:tmax]),
    x ∈ Interval(config[:xmin], config[:xmax]), y ∈ Interval(config[:ymin], config[:ymax])]
# Discretization
dx = config[:dx]

In [ ]:
#v1(t,x,y) = cos(pi*x)
#v2(t,x,y) = cos(pi*y) # with minus is divergence free

r0 = 0.16
r2(x,y) = (x-0.5)^2 + (y-0.5)^2
v1(t,x,y) = (x-0.5)*r2(x,y)*(r2(x,y) - r0)
v2(t,x,y) = (y-0.5)*r2(x,y)*(r2(x,y) - r0)



In [ ]:
xs = collect(config[:xmin]:0.01:config[:xmax])
ys = collect(config[:ymin]:0.01:config[:ymax])

v1_d = [(x-0.5)*r2(x,y)*(r2(x,y) - r0) for x in xs for y in ys]
v2_d = [(y-0.5)*r2(x,y)*(r2(x,y) - r0) for x in xs for y in ys]
pv1 = plot(ys, xs, v1_d, linetype = :contourf, title = "v1", aspect_ratio = 1)
pv2 = plot(ys, xs, v2_d, linetype = :contourf, title = "v2", aspect_ratio = 1)
plot(pv1, pv2, layout = (1,2))

In [ ]:
phi(x,y) = (x*(x-1)*y*(y-1))^4*2^(4p)

p=4
B1_d = [ p*y^(p-1)*(y-1)^(p-1)*(2y-1)*x^p*(x-1)^p*2^(4p) for x in xs for y in ys]
B2_d = [ -p*x^(p-1)*(x-1)^(p-1)*(2x-1)*y^p*(y-1)^p*2^(4p) for x in xs for y in ys]
pv1 = plot(xs, ys, B1_d, linetype = :contourf, title = "Initial B1", aspect_ratio = 1)
pv2 = plot(xs, ys, B2_d, linetype = :contourf, title = "Initial B2", aspect_ratio = 1)
plot(pv1, pv2, layout = (1,2))

In [ ]:

eqs = [Dt(B1(t,x,y)) ~ Dy(v2(t,x,y)*B1(t,x,y) - v1(t,x,y)*B2(t,x,y)), 
    Dt(B2(t,x,y)) ~ Dx(v1(t,x,y)*B2(t,x,y) - v2(t,x,y)*B1(t,x,y)), 
    Dx(B1(t,x,y)) + Dy(B2(t,x,y))  ~ 0]


### Boundary conditions

Using the constraint the equations are equivalent to:

$$
\partial_t B^i = v^j\partial_j B^i + l.o.t.
$$
Thus, at the boundaries we have to impose boundary conditions (non-incoming in our case) only when
$v^jn_j \geq 0$.

Otherwise we could put periodic boundary conditions by imposing for instance $B^i(t,0,y) = B^i(t,L,y)$


In [ ]:
non_incoming = false
periodic = false

#non_incoming = true
#periodic = true
if non_incoming 
bcs = [B1(0,x,y) ~ Dy(phi(x,y)),
    B2(0,x,y) ~ -Dx(phi(x,y)),
    (1 - sign(v1(t,0,y)))*sign(v1(t,0,y))*B1(t,0,y) ~ 0,
    (1 + sign(v1(t,1,y)))*sign(v1(t,1,y))*B1(t,1,y) ~ 0,
    (1 - sign(v2(t,x,0)))*sign(v2(t,x,0))*B1(t,x,0) ~ 0,
    (1 + sign(v2(t,x,1)))*sign(v2(t,x,1))*B1(t,x,1) ~ 0,
    (1 - sign(v1(t,0,y)))*sign(v1(t,0,y))*B2(t,0,y) ~ 0,
    (1 + sign(v1(t,1,y)))*sign(v1(t,1,y))*B2(t,1,y) ~ 0,
    (1 - sign(v2(t,x,0)))*sign(v2(t,x,0))*B2(t,x,0) ~ 0,
    (1 + sign(v2(t,x,1)))*sign(v2(t,x,1))*B2(t,x,1) ~ 0]

elseif periodic

    bcs = [B1(0,x,y) ~ Dy(phi(x,y)),
    B2(0,x,y) ~ -Dx(phi(x,y)),
    B1(t,0,y) ~ B1(t,1,y),
    B1(t,x,0) ~ B1(t,x,1),
    B2(t,0,y) ~ B2(t,1,y),
    B2(t,x,0) ~ B2(t,x,1)]

else
#simple boundary condition when V is outgoing everywhere
    #=
    bcs = [B1(0,x,y) ~ Dy(phi(x,y)),
    B2(0,x,y) ~ -Dx(phi(x,y))]
    =#
    bcs = [B1(0,x,y) ~ 4y^3*(y-1)^3*x^4*(x-1)^4*(2y-1)*2^(16),
    B2(0,x,y) ~ -4x^3*(x-1)^3*y^4*(y-1)^4*(2x-1)*2^(16)]
end


In [ ]:
# Neural network
input_ = length(domains)
n = config[:N_neurons]
#chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, 1)) for _ in 1:2] #this work well without GPU
chain = Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, n, σ), Dense(n, 2))

#strategy = QuadratureTraining()
strategy = QuasiRandomTraining(config[:N_points]; bcs_points = config[:BCS_points], minibatch = config[:Minibatch])
#strategy = GridTraining(dx)
ps = Lux.setup(Random.default_rng(), chain)[1] 
ps = ps |> ComponentArray |> gpud .|> Float64
#discretization = PhysicsInformedNN(chain, strategy) 
discretization = PhysicsInformedNN(chain, strategy; init_params = ps)

@named MHD_toy = PDESystem(eqs, bcs, domains, [t, x, y], [B1(t, x, y), B2(t, x, y)])
prob = discretize(MHD_toy, discretization)
sym_prob = symbolic_discretize(MHD_toy, discretization)

pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

loss = Float64[]
#loss_pde = []
#loss_bcs = []

callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    push!(loss, l)
    #push!(loss_pde, map(l_ -> l_(p.u), pde_inner_loss_functions))
    #push!(loss_bcs, map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end


In [ ]:

#res = Optimization.solve(prob, BFGS(linesearch = BackTracking()); maxiters = config[:maxiters], callback)
res = Optimization.solve(prob, config[:optimizer]; maxiters = config[:maxiters], callback)


In [ ]:
plot(loss, yscale = :log10, title = "Loss history", xlabel = "Iteration", ylabel = "Loss")

In [ ]:
phi_d = discretization.phi

In [ ]:
#depvars = [:B1, :B2]
#minimizers_ = [res.u.depvar[depvars[i]] for i in 1:length(chain)]

resu = res.u
@save "toy_MHD_2D.jld2" config domains eqs bcs resu loss

In [ ]:
xs, ys = [0.0:0.01:1.0 for d in 1:2]
#depvars = [:B1, :B2]
#minimizers_ = [res.u.depvar[depvars[i]] for i in 1:length(chain)]


In [ ]:
t_p = config[:tmax]

#u_predict = [[phi_d([t_p, x, y], res.u[i])[1] for x in xs for y in ys] for i in 1:2]
u_predict = [[Array(phi_d([t_p, x, y], res.u))[i] for x in xs for y in ys] for i in 1:2]

ps = []
for i in 1:2
    p1 = plot(xs, ys, u_predict[:,:][i], 
        linetype = :contourf, 
        #st = :surface,
        title = "predict B$i", aspect_ratio = 1)
    push!(ps, p1)
end

plot(ps[1], ps[2], layout = (1,2))